In [1]:
import cv2
import os


In [2]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import easyocr

In [3]:
def preprocess_image(image_path):   
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    
    return binary

In [4]:
def yolo_to_pixel(image, box):
    image_height, image_width = image.shape[:2]
    x_center, y_center, width, height = box
    x_min = int((x_center - width / 2) * image_width)
    y_min = int((y_center - height / 2) * image_height)
    x_max = int((x_center + width / 2) * image_width)
    y_max = int((y_center + height / 2) * image_height)
    return (x_min, y_min, x_max, y_max)

In [5]:
def crop_boxes(image, boxes, output_dir):
    cropped_images = []
    for i, box in enumerate(boxes):
        x_min, y_min, x_max, y_max = box
        cropped_image = image[y_min:y_max, x_min:x_max]
        output_path = os.path.join(output_dir, f"cropped_{i}.jpg")
        cv2.imwrite(output_path, cropped_image)
        cropped_images.append(cropped_image)
    return cropped_images

In [6]:
def perform_ocr(images):
    reader = easyocr.Reader(['fr'], gpu=False)  
    ocr_results = []
    for img in images:
        result = reader.readtext(img)
        ocr_results.append(result)
    return ocr_results

In [7]:
yolo_boxes = [
    (0.177232, 0.327731, 0.331250, 0.053221),
    (0.212500, 0.454482, 0.410714, 0.057423),
    (0.319643, 0.528711, 0.173214, 0.054622),
    (0.215625, 0.655462, 0.377679, 0.058824),
    (0.821429, 0.803221, 0.183929, 0.057423)
]

image_path = r'C:\Users\pc\Desktop\lp BigData\s6\moroccan_idCard_scanner\ids\old_card\cni-recto.jpg'
image = cv2.imread(image_path)


pixel_boxes = [yolo_to_pixel(image, box) for box in yolo_boxes]

dir = r"C:\Users\pc\Desktop\lp BigData\s6\moroccan_idCard_scanner\ids\cropped"

cropped_images = crop_boxes(image, pixel_boxes, dir)

In [11]:
res = perform_ocr(cropped_images)

Using CPU. Note: This module is much faster with a GPU.


In [23]:
from datetime import datetime

def transform_date(input_date):
    input_date = input_date.replace(',', '.')
    
    try:
        date_obj = datetime.strptime(input_date, '%d.%m.%Y')
    except ValueError:
        try:
            date_obj = datetime.strptime(input_date, '%d,%m,%Y')
        except ValueError:
            return "Invalid date format"
    
    formatted_date = date_obj.strftime('%d-%m-%Y')
    
    return formatted_date

In [25]:
data_dict = {}

data_dict['firstName'] = res[0][0][1] 
data_dict['secondName'] = res[1][0][1] 
data_dict['DOB'] = res[2][0][1] 
data_dict['COB'] = res[4][0][1] 

data_dict['DOB'] = transform_date(data_dict['DOB'])

print(data_dict)

{'firstName': 'OUSSAMA', 'secondName': 'NAJEM', 'DOB': '17-11-2003', 'COB': 'Q359550'}
